# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233951670730                   -0.50    7.0         
  2   -7.249981558942       -1.80       -1.38    1.0   7.39ms
  3   -7.250970783128       -3.00       -1.85    2.0   9.32ms
  4   -7.251074973000       -3.98       -1.94    1.0   7.71ms
  5   -7.251321278380       -3.61       -2.55    1.0   7.32ms
  6   -7.251337361146       -4.79       -3.05    2.0   8.32ms
  7   -7.251338576034       -5.92       -3.41    2.0   8.86ms
  8   -7.251338787166       -6.68       -4.08    2.0   8.84ms
  9   -7.251338797879       -7.97       -4.60    2.0   9.53ms
 10   -7.251338798557       -9.17       -5.02    2.0   8.52ms
 11   -7.251338798694       -9.86       -5.56    2.0   8.81ms
 12   -7.251338798702      -11.10       -5.96    1.0   7.63ms
 13   -7.251338798704      -11.76       -6.19    2.0   9.09ms
 14   -7.251338798704      -12.45       -6.63    1.0   7.73ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04524576679943123
[ Info: Arnoldi iteration step 2: normres = 0.6463516055781104
[ Info: Arnoldi iteration step 3: normres = 0.8255046496442345
[ Info: Arnoldi iteration step 4: normres = 0.3369889905501869
[ Info: Arnoldi iteration step 5: normres = 0.2911195699616518
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.92e-02, 5.25e-02, 1.82e-01, 2.18e-01, 1.34e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5651185455187402
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.33e-02, 1.41e-01, 5.19e-01, 1.04e-01, 1.37e-01)
[ Info: Arnoldi iteration step 7: normres = 0.053755117253770274
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.09e-04, 2.27e-02, 9.27e-03, 1.67e-02, 3.67e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10553694594507057
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.86e-